In [10]:
import PyPDF2
# Для анализа структуры PDF и извлечения текста
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure
# Для извлечения текста из таблиц в PDF
import pdfplumber
# Для извлечения изображений из PDF
from PIL import Image
from pdf2image import convert_from_path
# Для выполнения OCR, чтобы извлекать тексты из изображений 
import pytesseract 
# Для удаления дополнительно созданных файлов
import os

import csv

In [11]:
 #Создаём функцию для извлечения текста
def text_extraction(element):
    # Извлекаем текст из вложенного текстового элемента
    line_text = element.get_text()
    
    # Находим форматы текста
    # Инициализируем список со всеми форматами, встречающимися в строке текста
    line_formats = []
    for text_line in element:
        if isinstance(text_line, LTTextContainer):
            # Итеративно обходим каждый символ в строке текста
            for character in text_line:
                if isinstance(character, LTChar):
                    # Добавляем к символу название шрифта
                    line_formats.append(character.fontname)
                    # Добавляем к символу размер шрифта
                    line_formats.append(character.size)
    # Находим уникальные размеры и названия шрифтов в строке
    format_per_line = list(set(line_formats))
    
    # Возвращаем кортеж с текстом в каждой строке вместе с его форматом
    return (line_text, format_per_line)

In [12]:
def is_header(text: str) -> bool:
    # Реализуйте здесь свои эвристики для определения заголовка
    # Например, вы можете проверить, начинается ли текст с заглавной буквы
    # или содержит ли он слова, которые часто используются в заголовках
    return text.isupper() or any(word in text.lower() for word in ['chapter', 'section', 'heading'])


In [15]:
#Разделяю текст на преддожения
pdf_path = 'instrukciya-po-ehkspluatacii-v8.pdf'

output_folder = '../data/extracted_files/doctorexplaindocument/'

# создаём объект файла PDF
pdfFileObj = open(pdf_path, 'rb')
# создаём объект считывателя PDF
pdfReaded = PyPDF2.PdfReader(pdfFileObj)

# Создаём словарь для извлечения текста из каждого изображения
text_per_page = {}
# Извлекаем страницы из PDF
for pagenum, page in enumerate(extract_pages(pdf_path)):
    
    # Инициализируем переменные, необходимые для извлечения текста со страницы
    pageObj = pdfReaded.pages[pagenum]
    page_text = []
    headers=[]
    line_format = []
    text_from_images = []
    text_from_tables = []
    page_content = []
    # Инициализируем количество исследованных таблиц
    table_num = 0
    first_element= True
    table_extraction_flag= False
    # Открываем файл pdf
    pdf = pdfplumber.open(pdf_path)
    # Находим исследуемую страницу
    page_tables = pdf.pages[pagenum]
    # Находим количество таблиц на странице
    tables = page_tables.find_tables()


    # Находим все элементы
    page_elements = [(element.y1, element) for element in page._objs]
    # Сортируем все элементы по порядку нахождения на странице
    page_elements.sort(key=lambda a: a[0], reverse=True)

    # Находим элементы, составляющие страницу
    for i,component in enumerate(page_elements):
        # Извлекаем положение верхнего края элемента в PDF
        pos= component[0]
        # Извлекаем элемент структуры страницы
        element = component[1]

        # Проверяем, является ли элемент текстовым
        if isinstance(element, LTTextContainer):
            # Проверяем, находится ли текст в таблице
            if table_extraction_flag == False:
                # Используем функцию извлечения текста и формата для каждого текстового элемента
                (line_text, format_per_line) = text_extraction(element)
                # Добавляем текст каждой строки к тексту страницы
                page_text.append(line_text)
                # Добавляем формат каждой строки, содержащей текст
                line_format.append(format_per_line)
                page_content.append(line_text)
            if is_header(line_text):
                
                headers.append(line_text)
            else:
                # Пропускаем текст, находящийся в таблице
                pass

 

            # Проверяем, извлекли ли мы уже таблицы из этой страницы
            # if element.y0 >= lower_side and element.y1 <= upper_side:
            #     pass
            if i + 1 < len(page_elements):
              if not isinstance(page_elements[i+1][1], LTRect):
               table_extraction_flag = False
               first_element = True
               table_num += 1


    # Создаём ключ для словаря
    dctkey = 'Page_'+str(pagenum)
    # Добавляем список списков как значение ключа страницы
    text_per_page[dctkey]= [page_text, line_format, text_from_images,text_from_tables, page_content,headers]

# Закрываем объект файла pdf
pdfFileObj.close()

# Удаляем созданные дополнительные файлы
# os.remove('PDF_image.png')

# Удаляем содержимое страницы
result = ''.join(text_per_page['Page_0'][4])

#Весь текст в одну переменную
all_text = ""

for page_content in text_per_page.values():
    page_text = page_content[4]  # Получаем содержимое страницы
    all_text += ' '.join(page_text) + '\n'  # Добавляем текст страницы к all_text с добавлением символа новой строки

# Выводим результат
print(all_text)


 
  
  
  
  
 Инструкция  
по эксплуатации 
"1С:ПРЕДПРИЯТИЕ 8" 
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
 Москва 
Фирма «1С» 
2020 

ПРАВО ТИРАЖИРОВАНИЯ  
ПРОГРАММНЫХ СРЕДСТВ И ДОКУМЕНТАЦИИ 
ПРИНАДЛЕЖИТ ФИРМЕ «1С» 
Приобретая систему «1С:Предприятие»,  
вы тем самым даете согласие не допускать 
копирования программ и документации 
без письменного разрешения фирмы «1С» 
 © ООО «1С», 2020 
Фирма «1С», Москва, 123056, а/я 64. 
Отдел продаж: Селезневская ул., 21, 
телефон: (495) 737-92-57, 
факс: (495) 681-44-07. 
E-mail: 1c@1c.ru 
URL: www.1c.ru, www.v8.1c.ru 
Группа разработки программ – А. Алексеев, А. Безбородов, Д. Бескоровайнов, 
П. Василец,  А. Виноградов,  А. Волков,  И. Гольштейн,  Е. Горностаев, 
Г. Дамье,  А. Даровских,  О. Дерут,  Н. Евграфов,  Б. Евтифеев,  Д. Зарецкий, 
Д. Ивашов,  С. Копиенко,  Н. Корсаков,  С. Кравченко,  В.  Кудрявцев, 
П. Кукушкин, А. Лакутин, М. Лейбович, Г. Леонтьев, А. Лехан, А. Макеев, 
А. Медведев,  Е. Митрошкин,  С. Мурзин,  С. Нуралие

In [29]:
for page_num, page_content in text_per_page.items():
    print(f"Page {page_num}:")
    for line in page_content[0]:
        print(line)

Page Page_0:
 

 

 

 

 

Инструкция  
по эксплуатации 
"1С:ПРЕДПРИЯТИЕ 8" 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

Москва 
Фирма «1С» 
2020 

Page Page_1:
ПРАВО ТИРАЖИРОВАНИЯ  
ПРОГРАММНЫХ СРЕДСТВ И ДОКУМЕНТАЦИИ 
ПРИНАДЛЕЖИТ ФИРМЕ «1С» 
Приобретая систему «1С:Предприятие»,  
вы тем самым даете согласие не допускать 
копирования программ и документации 
без письменного разрешения фирмы «1С» 

© ООО «1С», 2020 
Фирма «1С», Москва, 123056, а/я 64. 
Отдел продаж: Селезневская ул., 21, 
телефон: (495) 737-92-57, 
факс: (495) 681-44-07. 
E-mail: 1c@1c.ru 
URL: www.1c.ru, www.v8.1c.ru 
Группа разработки программ – А. Алексеев, А. Безбородов, Д. Бескоровайнов, 
П. Василец,  А. Виноградов,  А. Волков,  И. Гольштейн,  Е. Горностаев, 
Г. Дамье,  А. Даровских,  О. Дерут,  Н. Евграфов,  Б. Евтифеев,  Д. Зарецкий, 
Д. Ивашов,  С. Копиенко,  Н. Корсаков,  С. Кравченко,  В.  Кудрявцев, 
П. Кукушкин, А. Лакутин, М. Лейбович, Г. Леонтьев, А. Лехан, А. Макеев, 
А. Медведев,  Е. Митрошки

In [36]:
print(text_per_page['Page_10'])

[['Введение \n', 'Настоящая  книга  является  Руководством  по  использованию  системы \n«1С:Предприятие» (далее – Руководство). \nпользователей \nДанное \nпредназначено \n«1С:Предприятия»,  ответственных \nсистемой,  – \nбухгалтеров,  расчетчиков,  так  и  для  пользователей,  работающих  с \nограниченным  набором  функций  программы, \nоператоров, \nменеджеров, руководителей. \nТак  как  работа  системы  во  многом  определяется  используемой \nконфигурацией,  для \nработы \nконкретных  документов,  отчетов  и  других  элементов  конфигурации \nнеобходимо  изучить  описание  конфигурации,  которая  используется \nпри работе с «1С:Предприятием». \nСпециалистам, \nи \nадминистрированием  системы,  необходимо  обращаться  к  книгам \n«1С:Предприятие  8.3.  Руководство  разработчика»  и  «1С:Предприятие \n8.3. Руководство администратора». \n', 'для \nс \n', 'руководство \n', 'как \nза  работу \n', '– \n', 'особенностями \n', 'ознакомления \n', 'с \n', 'конфигурированием \n', 'занимающимс

In [ ]:
def table_of_contents():
  